In [1]:
import tensorflow as tf

/anaconda3/envs/tf-cpu/lib/python3.5/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


<img src="optimizer_graph.png" width="200px">

## Reading Data

We are reading MNIST data from http://yann.lecun.com/exdb/mnist/

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz


Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## Setting hyper-parameters

In [5]:
# Hyper-parameters
noofdatapoints = mnist.train.num_examples

learningrate = 0.001
nepochs = 10
batch_size = 512
noofbatches = noofdatapoints//batch_size

nhidden1 = 256
nhidden2 = 256 
ninput = 784
noutput = 10

## Defining X and Y as placeholders

In [6]:
X = tf.placeholder(tf.float32, [None, ninput])
Y = tf.placeholder(tf.float32, [None, noutput])

## Initializing Weights and Biases

In [7]:
weights = \
{
        'h1': tf.Variable(tf.random_normal([ninput, nhidden1])),
        'h2': tf.Variable(tf.random_normal([nhidden1, nhidden2])),
        'out': tf.Variable(tf.random_normal([nhidden2, noutput]))
}

biases = \
{
    'b1': tf.Variable(tf.random_normal([nhidden1])),
    'b2': tf.Variable(tf.random_normal([nhidden2])),
    'out': tf.Variable(tf.random_normal([noutput]))
}

## Defining model using sigmoid activation function

In [8]:
def multiperceptron(x):
    l1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    l2 = tf.nn.sigmoid(tf.add(tf.matmul(l1, weights['h2']), biases['b2']))
    outl = tf.nn.sigmoid(tf.add(tf.matmul(l2, weights['out']), biases['out']))
    return outl
    
model = multiperceptron(X)

## Define cost, optimizer and accuracy

In [9]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learningrate)
train_min = optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

## Running the session 
This will run the graph and use all the tensors that were previously defined

In [10]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(nepochs):
        for _ in range(noofbatches):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Use training data for optimization
            sess.run(train_min, feed_dict={X:batch_x, Y:batch_y})
        # Validate after every epoch
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        losscalc, accuracycalc = sess.run([loss, accuracy], feed_dict={X:batch_x, Y:batch_y})
        print("Epoch: %d, Loss: %0.4f, Accuracy: %0.4f"%(epoch, losscalc, accuracycalc))
            
    # When the training is complete and you are happy with the result
    accuracycalc = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels})
    print("Testing accuracy: %0.4f"%(accuracycalc))

Epoch: 0, Loss: 1.9023, Accuracy: 0.5469


Epoch: 1, Loss: 1.7783, Accuracy: 0.6621


Epoch: 2, Loss: 1.7096, Accuracy: 0.7422


Epoch: 3, Loss: 1.7222, Accuracy: 0.7207


Epoch: 4, Loss: 1.7196, Accuracy: 0.7305


Epoch: 5, Loss: 1.7292, Accuracy: 0.7070


Epoch: 6, Loss: 1.6769, Accuracy: 0.7559


Epoch: 7, Loss: 1.7022, Accuracy: 0.7344


Epoch: 8, Loss: 1.6975, Accuracy: 0.7324


Epoch: 9, Loss: 1.6703, Accuracy: 0.7656
Testing accuracy: 0.7479
